In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/My Drive/colab/eee/rotten.csv")

In [ ]:
df.head()

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,beavers,1145982,2003-05-23,Ivan M. Lincoln,False,3.5/4,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,http://www.deseretnews.com/article/700003233/B...
1,blood_mask,1636744,2007-06-02,The Foywonder,False,1/5,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,http://www.dreadcentral.com/index.php?name=Rev...
2,city_hunter_shinjuku_private_eyes,2590987,2019-05-28,Reuben Baron,False,NaN,fresh,CBR,The choreography is so precise and lifelike at...,POSITIVE,https://www.cbr.com/city-hunter-shinjuku-priva...
3,city_hunter_shinjuku_private_eyes,2558908,2019-02-14,Matt Schley,False,2.5/5,rotten,Japan Times,The film's out-of-touch attempts at humor may ...,NEGATIVE,https://www.japantimes.co.jp/culture/2019/02/0...
4,dangerous_men_2015,2504681,2018-08-29,Pat Padua,False,NaN,fresh,DCist,Its clumsy determination is endearing and some...,POSITIVE,http://dcist.com/2015/11/out_of_frame_dangerou...


In [ ]:
df.shape

(1444963, 11)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        tokens = word_tokenize(text.lower())  # Tokenize the text and convert to lowercase
        tokens = [token for token in tokens if token.isalpha()]  # Filter out non-alphabetic tokens
        tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
        tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize tokens
        return ' '.join(tokens)  # Return preprocessed text as a string
    else:
        return ''

In [ ]:
df['clean_text'] = df['reviewText'].apply(preprocess_text)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_text'])

In [ ]:
num_clusters = 3  # Number of clusters (can be adjusted)
kmeans = KMeans(n_clusters=num_clusters, random_state=42)  # Initialize KMeans model
kmeans.fit(tfidf_matrix)  # Fit KMeans model to the TF-IDF matrix
df['cluster'] = kmeans.labels_  # Assign cluster labels to the DataFrame

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import numpy as np

# Sample a subset of the data for evaluation
sample_size = 1000  # Adjust the sample size as needed
sample_indices = np.random.choice(df.shape[0], sample_size, replace=False)
sample_tfidf_matrix = tfidf_matrix[sample_indices]
sample_labels = kmeans.labels_[sample_indices]

# Calculate the Silhouette Score for the sample
sample_silhouette_avg = silhouette_score(sample_tfidf_matrix, sample_labels)
print("Silhouette Score :", sample_silhouette_avg)


Silhouette Score : 0.015897626192846927
